In [1]:
import sys
sys.path.append('/cluster/sj1/bb_opt/scripts')

In [2]:
%load_ext autoreload
%autoreload 2

import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import collect_stats as cs

%matplotlib inline

In [3]:
import collect_stats as cs

In [11]:
exp_folder = '/cluster/sj1/bb_opt/experiments/imdbwiki'
batches = [50]
num_acks = 0

In [5]:
filenames = ['wiki']

In [6]:
def sigmoid(x, exp=np.exp):
  return 1.0 / (1.0 + exp(-x))

1. ensemble16 - verify paper results

In [7]:
arrs = {}

In [47]:
to_read = {
    'ensemble16/o_max_std_start_3k_reg_': [None, 'normal3'],
    #'ensemble18/o_max_std_start_3k_reg_': [None, 'normal'],
    #'ensemble18/o_max_std_start_3k_reg2_': [None, 'normal2'],
    #'ensemble18/o_max_std_start_3k_reg_uinput_maxvar_g000.10.50102_': [None, 'MOD_small'],
    #'ensemble18/o_max_std_start_3k_reg_uinput_maxvar_g00020510204080_': [None, 'MOD_big'],
}


In [48]:
for k in to_read:
    arrs[k] = to_read[k]
for experiment in to_read:
    print('reading', experiment)
    arrs[experiment][0] = cs.get_data(exp_folder, experiment, batches, num_samples=20)

reading ensemble16/o_max_std_start_3k_reg_
reading sample 1
reading sample 2
reading sample 3
reading sample 4
reading sample 5
reading sample 6
reading sample 7
reading sample 8
reading sample 9
reading sample 10
reading sample 11
reading sample 12
reading sample 13
reading sample 14
reading sample 15
reading sample 16
reading sample 17
reading sample 18
reading sample 19
reading sample 20


In [49]:
to_eval = [
    'ensemble16/o_max_std_start_3k_reg_',
    'ensemble18/o_max_std_start_3k_reg_',
    'ensemble18/o_max_std_start_3k_reg2_',
    'ensemble18/o_max_std_start_3k_reg_uinput_maxvar_g000.10.50102_',
    'ensemble18/o_max_std_start_3k_reg_uinput_maxvar_g00020510204080_',
]

In [16]:
test_fn = lambda bs, prop: lambda x, filename : x[bs + 'test_pred_stats'][prop]
bs_test_rmse_fn = lambda x, filename : x['test_pred_stats']['baseline_rmse']
ood_fn = lambda bs, prop: lambda x, filename : x[bs + 'ood_pred_stats'][prop]
bs_ood_rmse_fn = lambda x, filename : x['test_pred_stats']['baseline_rmse']
test_log_fn = lambda prop, prop2: lambda x, filename : x['logging'][1][prop][prop2]

In [21]:
import collect_stats as cs

In [50]:
dist = "test"
dist = "ood"
prop = "log_prob"
#prop = "rmse"
#prop = "pred_std"
#prop = 'rmse_std_corr'
bs = "zero_gamma_"
bs = ""
#bs = "invar_gamma_"

#fn = [globals()[dist+"_" + prop + "_fn"](""),globals()[dist + "_" + prop + "_fn"](bs)]
fn = [globals()[dist+"_fn"]("", prop), globals()[dist+"_fn"](bs, prop)]
cs.prop_test2(50, 
             filenames,
             fn,
             arrs,
             to_eval[0],
             0,
             pval_threshold=1,
             paired_test=False,
            )

comparing <lambda> <lambda>
wiki 1.0 -0.13049787059426307 -0.13049787059426307 0 	 (std: 0.04777 0.04777, #n: 20 20)
combined pval: 1.00000 vs nan
count: 0/1


In [18]:
cs.prop_test(50, 
             filenames,
             test_fn("", "log_prob"), 
             arrs, 
             [to_eval[3], to_eval[5]],
             0,
             pval_threshold=1, 
             paired_test=False,
            )

comparing uninput_lowgamma_noise_1 bensemble
wiki 0.0701640609709938 -0.24285537302494048 -0.2511301629245281 0 	 (std: 0.01383 0.01354, #n: 20 20)
combined pval: 0.07016 vs nan
count: 0/1


In [ ]:
cs.plot_data_vs_ack_iter(
    50,
    filenames,
    'avg_seeds',
    'rmse',
    test_rmse_fn,
    20,
    arrs,
    to_eval,
    legend_loc=3,
)

In [ ]:
cs.plot_data_vs_ack_iter(
    20,
    filenames,
    'avg_seeds',
    'log_prob',
    lambda x, filename: x['best_gamma'],
    30,
    arrs,
    to_eval,
    legend_loc=3,
)

In [ ]:
cs.plot_data_vs_ack_iter(
    50,
    filenames,
    'avg_seeds',
    'nll',
    lambda x, filename : x['logging'][1]['best']['nll'],
    20,
    arrs,
    to_eval,
    legend_loc=3,
)

In [ ]:
cs.plot_data_vs_ack_iter(
    200,
    filenames,
    'avg_seeds',
    'rmse',
    test_rmse_fn,
    20,
    arrs,
    to_eval,
    legend_loc=3,
)